# 1. Import package

In [0]:
!pip install surprise

    100% |████████████████████████████████| 3.3MB 10.4MB/s 
  Running setup.py bdist_wheel for scikit-surprise ... - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
import pandas as pd

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.accuracy import mae
from surprise.accuracy import rmse

from surprise import SVD
from surprise import KNNBasic

# 2. Load Data

In [0]:
train_df = pd.read_csv('training.txt',header = None, sep = '\t')
test_df = pd.read_csv('test.txt', header = None, sep = '\t')
train_df.columns = ['userID','itemID','rating']
test_df.columns = ['userID','itemID','rating']

reader = Reader(rating_scale=(1, 5))
train = Dataset.load_from_df(train_df[['userID', 'itemID', 'rating']], reader)
test = Dataset.load_from_df(test_df[['userID', 'itemID', 'rating']], reader)

trainset = train.build_full_trainset()
testset = test.build_full_trainset()

In [0]:
anti_testset = testset.build_anti_testset()

# 3. Recommendation

In [0]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

## 3.1 SVD-based CF

In [0]:
algo_svd = SVD()
algo_svd.fit(trainset)

In [0]:
# predict ratings for all pairs (u, i) that are NOT in the training set.
predictions_svd_recommend = algo_svd.test(anti_testset)

In [0]:
recommend_top_20 = get_top_n(predictions_svd_recommend, n=20)

In [0]:
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [0]:
uid = 'A24FQNZ2ZCP9UH'
iid = 'B004DK0UDA'
pred = algo_svd.predict(uid, iid, r_ui=4, verbose=True)

user: A24FQNZ2ZCP9UH item: B004DK0UDA r_ui = 4.00   est = 4.51   {'was_impossible': False}


## 3.1 KNN-based CF

In [0]:
algo_knn = KNNBasic()
algo_knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [0]:
uid = 'A24FQNZ2ZCP9UH'
iid = 'B004DK0UDA'
pred = algo_knn.predict(uid, iid, r_ui=4, verbose=True)

user: A24FQNZ2ZCP9UH item: B004DK0UDA r_ui = 4.00   est = 4.76   {'actual_k': 40, 'was_impossible': False}


# 4. Evaluation

<img src="Precisionrecall.png" width="300">

In [0]:
from collections import defaultdict

def precision_recall_F1_HR(predictions, k = 20, threshold=3.5):

    # First map the predictions to each user.
    user_est_true = defaultdict(list) 
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
        # user_ear_true is a dictionary 
        # with key = uid (user id)
        # and value = [(est,true_r),(est,true_r),...]: a list of (est_rating, true_rating)

    precisions = dict()
    recalls = dict()
    hitrates = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items (TP + FN)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items (top k) (TP + FP)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k (TP)
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant TP/(TP + FP)
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended TP/(TP + FN)
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
        # HitRate@K: TP/(TP + TN + FP + FN)
        hitrates[uid] = n_rel_and_rec_k / len(user_ratings)
        
    
    P = sum(prec for prec in precisions.values()) / len(precisions)
    R = sum(rec for rec in recalls.values()) / len(recalls)
    HR = sum(hr for hr in hitrates.values()) / len(hitrates)
    print('Precision:',P)
    print('Recall:', R)
    print('F1 score:', 2 * P * R/ (P + R))
    print('Hit rate:', HR)

## 4.1 SVD-based CF

In [0]:
# predictions
predictions_svd_train = algo_svd.test(trainset.build_testset())
predictions_svd_test = algo_svd.test(testset.build_testset())

In [0]:
predictions_svd_test[0][1]

'B00161DDJM'

In [0]:
# precision, recall, F1, hit rate
precision_recall_F1_HR(predictions_svd_train, k = 20)
precision_recall_F1_HR(predictions_svd_test, k = 20)

Precision: 0.8490197929894814
Recall: 0.9462986599552646
F1 score: 0.8950237113238975
Hit rate: 0.7608483514673351
Precision: 0.7498017126729373
Recall: 0.7585513290366065
F1 score: 0.7541511435775713
Hit rate: 0.7468862543581362


In [0]:
# Mean Absolute Error
mae(predictions_svd_train) # train
mae(predictions_svd_test) # test

MAE:  0.5799
MAE:  0.8497


0.8497134373534223

In [0]:
# Root Mean Squared Error
rmse(predictions_svd_train) # train
rmse(predictions_svd_test) # test

RMSE: 0.7622
RMSE: 1.1073


1.1072543725194606

## 4.2 KNN-based CF

In [0]:
# predictions
predictions_knn_train = algo_knn.test(trainset.build_testset())
predictions_knn_test = algo_knn.test(testset.build_testset())

In [0]:
# precision, recall, F1, hit rate
precision_recall_F1_HR(predictions_knn_train, k = 20)
precision_recall_F1_HR(predictions_knn_test, k = 20)

Precision: 0.9027194453167956
Recall: 0.9574286798259986
F1 score: 0.9292695297763668
Hit rate: 0.7653271340756546
Precision: 0.7375483682668064
Recall: 0.7445042215040951
F1 score: 0.7410099716137286
Hit rate: 0.7327131189857085


In [0]:
# Mean Absolute Error
mae(predictions_knn_train)
mae(predictions_knn_test)

MAE:  0.3038
MAE:  0.9141


0.9140914756042887

In [0]:
# Root Mean Squared Error
rmse(predictions_knn_train) # train
rmse(predictions_knn_test) # test

RMSE: 0.4410
RMSE: 1.2404


1.2403941971547823